In [1]:
from datasets import load_dataset

/Users/leo/miniconda3/envs/311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import csv
import re
import sys

csv.field_size_limit(sys.maxsize)

path = "/Users/leo/Desktop/Code/trade/news/fnspid_data/Stock_news/nasdaq_exteral_data.csv"
out_path = "/Users/leo/Desktop/Code/trade/news/fnspid_data/Stock_news/output.csv"

# Regex: Fast-check for valid row start (ID, Year-)
row_start_pattern = re.compile(r'^\d+\.\d+,202[0-3]-')

print("🚀 Starting ultra-fast extraction (Date, Source, Title, Link)...")

counter = 0
matches = 0

with open(path, 'r', encoding='utf-8', errors='ignore') as f_in, \
     open(out_path, 'w', encoding='utf-8', newline='') as f_out:
    
    writer = csv.writer(f_out)
    # New Header
    writer.writerow(['Date', 'Source', 'Title', 'Link'])
    
    next(f_in) # Skip original header
    
    for line in f_in:
        counter += 1
        
        # 1. Skip non-row lines immediately
        if not row_start_pattern.match(line):
            continue
            
        try:
            # 2. Slice to 8000 chars (Source is a bit further down than Link)
            # This avoids loading the massive "Article" or "Summary" text fields
            short_line = line[:8000] 
            row = next(csv.reader([short_line]))
            
            # Column Map: Date=1, Source=5, Title=2, Link=4
            if len(row) > 5:
                writer.writerow([row[1], row[5], row[2], row[4]])
                matches += 1
                
        except (csv.Error, StopIteration):
            continue
            
        if counter % 1000000 == 0:
            print(f"Scanned: {counter//1000000}M lines | Found: {matches:,} valid entries")

print(f"---")
print(f"DONE! Saved {matches:,} rows with metadata to {out_path}")

🚀 Starting ultra-fast slice-processing...
Scanned: 63M lines | Found: 1,527,544 entries
---
DONE! Saved 1,566,360 rows to /Users/leo/Desktop/Code/trade/news/fnspid_data/Stock_news/output.csv


In [5]:
df = pd.read_csv("/Users/leo/Desktop/Code/trade/news/fnspid_data/Stock_news/output.csv")

In [6]:
df

,Title,Link
0,Interesting A Put And Call Options For August ...,https://www.nasdaq.com/articles/interesting-a-...
1,Wolfe Research Initiates Coverage of Agilent T...,https://www.nasdaq.com/articles/wolfe-research...
2,Agilent Technologies Reaches Analyst Target Price,https://www.nasdaq.com/articles/agilent-techno...
3,Agilent (A) Enhances BioTek Cytation C10 With ...,https://www.nasdaq.com/articles/agilent-a-enha...
4,"Pre-Market Most Active for Dec 7, 2023 : SQQQ,...",https://www.nasdaq.com/articles/pre-market-mos...
...,...,...
1566355,Validea's Top Five Healthcare Stocks Based On ...,https://www.nasdaq.com/articles/valideas-top-f...
1566356,Validea's Top Five Healthcare Stocks Based On ...,https://www.nasdaq.com/articles/valideas-top-f...
1566357,"Zynex, Inc (ZYXI) Q4 2019 Earnings Call Transc...",https://www.nasdaq.com/articles/zynex-inc-zyxi...
1566358,Validea's Top Five Healthcare Stocks Based On ...,https://www.nasdaq.com/articles/valideas-top-f...
